# Aplicación para inferencia del modelo de segmentación semantica en video

(Opcional) Montar el acceso al espacio de Drive para el modelo y el video al cual se le ve aplicar la segmentación

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importar los paquetes necesarios para realizar la inferencia con el modelo.

In [2]:
import keras
from keras.models import *
from keras.layers import *

from types import MethodType
import random
import six
import json
from tqdm import tqdm
import cv2
import numpy as np
import itertools

Cargar el modelo. dentro de los parentesis poner el link a donde esta guardado el modelo con los pesos.

In [3]:
model=keras.models.load_model("/content/drive/Shareddrives/Trabajo de Grado-RECONOCIMIENTO DE OBJETOS Y ESTIMACION DE UN MAPA LOCAL PARA ´ PERSONAS CON IMPEDIMENTO VISUAL USANDO VISION POR COMPUTADOR./Documentos Trabajo de Grado/Modelo2/semasegUltimate1.keras")

Se define el tamaño del frame que va recibir el modelo y el tamaño de salida del frame al ejecutarse el modelo. Así como colores para pintar los objetos segmentados.

In [4]:
output_width = 112
output_height = 112
input_width = 224
input_height = 224
n_classes = 4
class_colors = [(random.randint(0, 255), random.randint(
    0, 255), random.randint(0, 255)) for _ in range(5000)]

Se define el método get_legends para poner las etiquetas con sus respectivos colores de los objetos segmentados.

In [5]:
def get_legends(class_names, colors=class_colors):

    n_classes = len(class_names)
    legend = np.zeros(((len(class_names) * 25) + 25, 125, 3),
                      dtype="uint8") + 255

    class_names_colors = enumerate(zip(class_names[:n_classes],
                                       colors[:n_classes]))

    for (i, (class_name, color)) in class_names_colors:
        color = [int(c) for c in color]
        cv2.putText(legend, class_name, (5, (i * 25) + 17),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(legend, (100, (i * 25)), (125, (i * 25) + 25),
                      tuple(color), -1)

    return legend

Se define el método concat_lenends tomando la imagen obtenida por el metodo get_legends para añadirla al frame de salida del modelo.

In [6]:
def concat_lenends(seg_img, legend_img):

    new_h = np.maximum(seg_img.shape[0], legend_img.shape[0])
    new_w = seg_img.shape[1] + legend_img.shape[1]

    out_img = np.zeros((new_h, new_w, 3)).astype('uint8') + legend_img[0, 0, 0]

    out_img[:legend_img.shape[0], :  legend_img.shape[1]] = np.copy(legend_img)
    out_img[:seg_img.shape[0], legend_img.shape[1]:] = np.copy(seg_img)

    return out_img

Se define el método get_image_array para guardar la imagen en una variable tipo numpy normalizada para pasar por el modelo de segmentación.

In [7]:
def get_image_array(image_input, width, height, imgNorm="sub_mean",
                  ordering='channels_last'):
    """ Load image array from input """

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif  isinstance(image_input, six.string_types)  :
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_image_array: path {0} doesn't exist".format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_image_array: Can't process input type {0}".format(str(type(image_input))))

    if imgNorm == "sub_and_divide":
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img/255.0

    if ordering == 'channels_first':
        img = np.rollaxis(img, 2, 0)
    return img

Se define el método visualize_segmentation para aplicar la operación del método get_colored_segmentation_image para aplicar la representación de colores para cada objeto segmentado.

In [8]:
def visualize_segmentation( seg_arr , inp_img=None  , n_classes=None ,
    colors=class_colors , class_names=None , overlay_img=False , show_legends=False ,
    prediction_width=None , prediction_height=None  ):


    if n_classes is None:
        n_classes = np.max(seg_arr)

    seg_img = get_colored_segmentation_image( seg_arr  , n_classes , colors=colors )

    if not inp_img is None:
        orininal_h = inp_img.shape[0]
        orininal_w = inp_img.shape[1]
        seg_img = cv2.resize(seg_img, (orininal_w, orininal_h))


    if (not prediction_height is None) and  (not prediction_width is None):
        seg_img = cv2.resize(seg_img, (prediction_width, prediction_height ))
        if not inp_img is None:
            inp_img = cv2.resize(inp_img, (prediction_width, prediction_height ))


    if overlay_img:
        assert not inp_img is None
        seg_img = overlay_seg_image( inp_img , seg_img  )


    if show_legends:
        assert not class_names is None
        legend_img = get_legends(class_names , colors=colors )

        seg_img = concat_lenends( seg_img , legend_img )


    return seg_img

Se define el método get_colored_segmentation_image para guardar el frame como un numpy tipo uint8.

In [9]:
def get_colored_segmentation_image( seg_arr  , n_classes , colors=class_colors ):
    output_height = seg_arr.shape[0]
    output_width = seg_arr.shape[1]

    seg_img = np.zeros((output_height, output_width, 3))

    for c in range(n_classes):
        seg_img[:, :, 0] += ((seg_arr[:, :] == c)*(colors[c][0])).astype('uint8')
        seg_img[:, :, 1] += ((seg_arr[:, :] == c)*(colors[c][1])).astype('uint8')
        seg_img[:, :, 2] += ((seg_arr[:, :] == c)*(colors[c][2])).astype('uint8')

    return seg_img


Permite cargar el video al cual se va a predecir con el modelo de segmentación semántica. Dentro de cv2. VideoCapture se pone el path donde este el video de interés.
También se hace el cálculo de los FPS del video. Por último se configura un objeto que permite guardar cada frame calculado en un archivo de video de extención .mp4. El video utilizado es el que sale en el siguiente link https://www.videvo.net/es/video/grupo-de-colegas-multietnicos-que-tienen-una-reunion-de-equipo-y-se-pasan-una-pelota-de-tenis/1113020/.

In [10]:
capture = cv2.VideoCapture("/content/drive/Shareddrives/Trabajo de Grado-RECONOCIMIENTO DE OBJETOS Y ESTIMACION DE UN MAPA LOCAL PARA ´ PERSONAS CON IMPEDIMENTO VISUAL USANDO VISION POR COMPUTADOR./Documentos Trabajo de Grado/Model/_import_61ea45b28b2592.20292381.mov")
fps = capture.get(cv2.CAP_PROP_FPS)
frame=0
result = cv2.VideoWriter('output.mp4',
                         0x7634706d ,
                         fps, (input_width+125 ,input_height))

Para el video de este ejericio se pone los segundos que se quiere predecir por el modelo. El modelo fue entrenado con las clases ['background','cup','laptop','person']. Se pueden cambiar los nombres a lo que se considere logico respectando el orden del vector. Ya cuando termina el proceso puede descargarse el video con extención .mp4.

In [ ]:
segundos=8
while (frame<segundos*fps):
  capture.set(cv2.CAP_PROP_POS_FRAMES, frame)
  ret, inp = capture.read()
  x = get_image_array(inp, input_width, input_height, ordering='channels_last')
  pr = model.predict(np.array([x]))[0]
  pr = pr.reshape((output_height,  output_width, n_classes)).argmax(axis=2)
  seg_img = visualize_segmentation( pr , inp ,n_classes=n_classes , colors=class_colors
        , overlay_img=False ,prediction_width=input_width , prediction_height=input_height,
        class_names=['background','cup','laptop','person'], show_legends=True,)
  image1copy = np.uint8(seg_img)
  grayscale_image  = cv2.cvtColor(image1copy, cv2.COLOR_HSV2BGR)
  result.write(grayscale_image)
  frame=frame+1
  print(frame)
capture.release()
result.release()

1/1 [==============================] - 3s 3s/step
1
1/1 [==============================] - 1s 691ms/step
2
1/1 [==============================] - 1s 670ms/step
3
1/1 [==============================] - 1s 657ms/step
4
1/1 [==============================] - 1s 846ms/step
5
1/1 [==============================] - 1s 833ms/step
6
1/1 [==============================] - 1s 772ms/step
7
1/1 [==============================] - 1s 1s/step
8
1/1 [==============================] - 1s 1s/step
9
1/1 [==============================] - 1s 904ms/step
10
1/1 [==============================] - 1s 659ms/step
11
1/1 [==============================] - 1s 701ms/step
12
1/1 [==============================] - 1s 669ms/step
13
1/1 [==============================] - 1s 775ms/step
14
1/1 [==============================] - 1s 1s/step
15
1/1 [==============================] - 2s 2s/step
16
1/1 [==============================] - 1s 677ms/step
17
1/1 [==============================] - 1s 729ms/step
18
1/1 [===========